---
# 0. 라이브러리 
---

In [1]:
# 문장을 토큰화하는 모듈 설치
!pip3 install torchtext==0.10.0
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 2.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.9.0 which is incompatible.
Looking in indexes: https://pypi.org/

In [2]:
import os 
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator
import spacy
import numpy as np
import torch.nn.functional as F

import random
import math
from tqdm.notebook import tqdm

from torchsummary import summary as summary_
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

---
# 1. Data
----

In [4]:
# 문장을 토큰화하는 모델 로드
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [5]:
# tokenizer function 생성
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

#### Field 정의

- sequential : 시퀀스 데이터 여부. (True가 기본값)
- use_vocab : 단어 집합을 만들 것인지 여부. (True가 기본값)
- tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
- lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
- batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
- is_target : 레이블 데이터 여부. (False가 기본값)
- fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행된다.

In [6]:
# torchtext의 Field는 데이터를 어떻게 처리할지 조절합니다.
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

---
### 1-1. Data Load
----

In [7]:
# 30,000개의 영어, 독일, 프랑스어 문장을 포함합니다.
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(SRC,TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:02<00:00, 420kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 113kB/s] 


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 108kB/s] 


In [8]:
print('Train 개수:', len(train_data))
print('Test 개수:', len(test_data))
print('Valid 개수:', len(valid_data))

Train 개수: 29000
Test 개수: 1000
Valid 개수: 1014


In [9]:
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


---
### 1-2. Vocab 생성
----

- min_freq = 해당 인수를 사용하여 n회 이상 나타내는 토큰만 어휘에 표시   
   n-1회 나타날 경우 <UNK\>으로 표시

In [10]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [11]:
print('Train Vocab의 수:', len(SRC.vocab))
print('Test Vocab의 수:', len(TRG.vocab))

Train Vocab의 수: 7853
Test Vocab의 수: 5893


In [12]:
batch_size = 128

# Iterator를 통해 원본 문장과 배치 문장과 동일하게 패딩 처리
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size=batch_size, device=device)

---
# 2. Model
----

---
### 2-1. Encoder
----

<img src='https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq8.png'>

- 양방향 Model을 사용

- 왼쪽에서 오른쪽으로 문장을 통과하는 순방향 Layer (아래 녹색)  
  context 벡터에서 마지막 단어를 본 후 순방향에 

- 오른쪽에서 왼쪽으로 문장을 통과하는 역방향 Layer (위 청색)  
  context 벡터에서 첫 번째 단어를 본후 역방향에 


In [13]:
from thinc.layers.bidirectional import bidirectional
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
    super().__init__()
    
    self.embedding = nn.Embedding(input_dim, emb_dim)
    self.gru = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True) # 각 레이어 사이에서 드롭아웃이 적용되기 때문에 여기서는 미적용

    self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
    '''enc_hid_dim * 2가 들어가는 이유는 Encoder의 GRU 계층이 양방향이기 때문
       즉, 입력 시퀀스를 양방향으로 모두 실행하고 각 방향에 대해 hidden state를 출력
       따라서, hidden state의 총 크기는 enc_hid_dim * 2와 동일하다.
       Linear 계층의 목적은 enc_hid_dim * 2 크기를 가진 Encoder의 GRU 계층에서 Decoder에 hidden state의 크기인 dec_hid_dim에 매핑하는 것'''

    self.dropout = nn.Dropout(dropout)

  def forward(self, src):
    out = self.embedding(src)  # src = [src_len , batch_size]
    embedded = self.dropout(out) # embedded = [src_len, batch_size, emb_dim]
                                 # src를 dense 벡터에 매핑하는 임베딩 레이어의 출력
    out, hidden = self.gru(embedded)
    # out = [src_len, barch_size, hid_dim] / LSTM의 출력
    # hidden = [n_layer, batch_size, hid_dim] / LSTM의 hidden state
    
    '''양방향 GRU 계층은 embedded input 시퀀스를 취하고 2개의 hidden state를 생성
       하나는 forward pass이고 다른 하나는 backward pass이다'''
    # hidden[-2, :, :] = 최종 time step(문장에서 마지막 단어를 본 후) 이후 최상위 계층의 순방향 GRU의 hidden state 
    # hidden[-1, :, :] = 최종 time step(문장에서 첫 번째 단어를 본 후) 이후 최상위 계층의 역방향 GRU의 hidden state

    hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

    return out, hidden  

    # out = [src_len, betch_size, enc_hid_dim * 2]
    # hidden = [batch_size, dec_hid_dim]    

---
### 2-2. Attention
----

<img src='https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq9.png'>

- Attention을 통해 Decoder의 이전 hidden state  
  Encoder에서 forward 또는 backward hidden state를 모두 포함하게 된다.

- 지금까지 디코딩한 것과 인코딩한 모든 것을  
  소스 문장에서 디코딩할 다음 단어를 정확하게 예측하기 위해   
  가장 주의를 기울여야 하는 단어를 나타내는 벡터를 생성

1. 이전 Decoder의 hidden state와 Encoder의 hidden state 사이의 에너지를 계산

2. Encoder의 hidden state = Tensro의 시퀀스   
   Decoder의 hidden state = 단일 Tensor

3. 이전 Decoder의 hidden state를 반복한 다음 에너지를 계산

4. Linear Layer와 뢀성화 함수를 사용해 사이를 연결

In [24]:
class Attention(nn.Module):
  def __init__(self, enc_hid_dim, dec_hid_dim):
    super().__init__()

    self.attention = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
    '''(enc_hid_dim * 2) + dec_hid_dim은 
       양방향 Encoder의 hidden state와 현재 Decoder의 hidden state의 연결에 있는 요소의 수를 의미'''
    self.v = nn.Linear(dec_hid_dim , 1, bias = False)

  def forward(self, hidden, encoder_outputs):

    batch_size = encoder_outputs.shape[1]
    src_len = encoder_outputs.shape[0]

    hidden = hidden.unsqueeze(1).repeat(1,src_len,1)
    '''hidden state를 반복하는 이유는 
       현재 대사 문장 단어를 기준으로 각 소스 문장 단어에 대해 Attention Score를 계산하기 위함
       Attention 매커니즘은 각 소스 문장 단어에 대한 Decoder의 현재 hidden state와 Encoder의 hidden state 사이의 유사성을 고려해 Score를 계산한다.
       Decoder와 Encoder 각각의 hidden state를 비교하기 위해 소스 문장의 길이를 따라서 Decoder의 hidden state를 반복한다.
       위 방법을 통해 각 소스 단어에 대한 Attention Score를 계산할 수 있게된다.'''

    encoder_outputs = encoder_outputs.permute(1, 0, 2)
    '''차원을 전치하기 위해 사용
       [src_len, batch_size, enc_hid_dim * 2] -> [batch_size, src_len, enc_hid_dim * 2]
       이후 hidden과 encoder_outputs를 concat하기 위해 정렬이 필요하기 때문'''
    energy = torch.tanh(self.attention(torch.cat((hidden, encoder_outputs), dim = 2))) # energy = [batch_size, src_len, dec_hid_dim]

    attentions = self.v(energy).squeeze(2) # attentions = [batch_size, src_len]

    return F.softmax(attentions, dim = 1)


---
### 2-3. Decoder
----

<img src='https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq10.png'>

In [25]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        '''dec_hid_dim = Decoder의 last hidden state
           enc_hid_dim = Weight context vector (Encoder가 양방향을 띄기 때문에 * 2를 해줌)
           emb_dim = embedded input'''

        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):

        input = input.unsqueeze(0) # input = [1, batch_size]
        
        embedded = self.dropout(self.embedding(input)) # embedded = [1, batch_size, emb_dim]
        
        a = self.attention(hidden, encoder_outputs) # a = [batch_size, src_len]
                
        a = a.unsqueeze(1) # a = [batch_size, 1, src_len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2) # encoder_outputs = [batch_size, src_len, enc_hid_dim * 2]
        
        # 3차원 Tensor의 행렬곱 [b x n x m] X [b x m x p] => [b x n x p]
        weighted = torch.bmm(a, encoder_outputs) # weighted = [batch_size, 1, enc_hid_dim * 2]
        
        weighted = weighted.permute(1, 0, 2) # weighted = [1, batch_size, enc_hid_dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)  # rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
        
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))   # output = [seq len, batch size, dec hid dim ]
                                                                    # hidden = [n layers , batch size, dec hid dim]
        
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0) 
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1)) # prediction = [batch_size, output_dim]
        
        return prediction, hidden.squeeze(0)

---
### 2-3. Seq2Seq
---

<img src='https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq7.png'>

1. input 문장 받기

2. Encoder를 이용해 context 벡터 생성 

3. Decoder를 이용해 예측 output 문장 생성 

- assert 함수

  - 가정 설정문 

  - 디버깅 목적으로 사용, 일반적으로 입력 인수의 유효성을 검사하거나 프로그램의 내부 상태를 검증하는데 사용

  - 특정 조건이 충족되는지 확인하고, 조건이 True이면 아무것도 발생X, 미충족시 AssertionError가 발생

In [26]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()

    self.encoder = encoder
    self.decoder = decoder 
    self.device = device

  def forward(self, src, trg, teacher_forcing_ratio = 0.5):
                  # src = [src_len, batch_size]
                  # trg = [trg_len, batch_size]

    batch_size = trg.shape[1]
    trg_len = trg.shape[0] # 타겟 토큰 길이 
    trg_vocab_size = self.decoder.output_dim # context vector의 차원
    
    # Decoder의 출력을 저장할 Tensor
    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

    encoder_outputs, hidden = self.encoder(src)
    # encoder_outputs는 양방향 입력 시퀀스의 hidden state
    # hidden은 Linear 레이어를 통과하는 최종 양방향 hidden state     

    # Decoder의 첫 번째 input을 <token>으로 사용
    input = trg[0,:]
    
    for i in range(1, trg_len):
      
      output, hidden = self.decoder(input, hidden, encoder_outputs)

      # 예측값 저장 
      outputs[i] = output

      # teacher forcing 사용 여부
      teacher_force = random.random() < teacher_forcing_ratio

      # 가장 높은 확률 값 얻기
      top1 = output.argmax(1)

      # teacher_forcing의 경우 다음 target token 입력
      input = trg[i] if teacher_force else top1

    return outputs


---
# 3. Model setting
----

In [27]:
# 하이퍼 파라미터 지정
input_dim = len(SRC.vocab)
output_dim = len(TRG.vocab)
enc_emb_dim = 256 # 임베딩 차원
dec_emb_dim = 256
enc_hid_dim = 512
dec_hid_dim = 512 # hidden state 차원
enc_dropout = 0.5
dec_dropout = 0.5

In [28]:
# 모델 생성
attn = Attention(enc_hid_dim, dec_hid_dim)
enc = Encoder(input_dim, enc_emb_dim, enc_hid_dim, dec_hid_dim, enc_dropout)
dec = Decoder(output_dim, dec_emb_dim, enc_hid_dim, dec_hid_dim, dec_dropout, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [29]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (gru): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attention): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5893, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [30]:
# 가중치 초기화
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

# 모델의 학습가능한 파라미터 수 측정
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainableparameters')

The model has 20,518,405 trainableparameters


In [31]:
# optimizer
optimizer = optim.Adam(model.parameters())

# loss function
# pad에 해당하는 index는 무시합니다.
trg_pad_idx = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)

---
# 4. Train
---

In [32]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(tqdm(iterator)):
        src = batch.src
        trg = batch.trg
        optimizer.zero_grad()

        output = model(src,trg) # [trg len, batch size, output dim]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim) # loss 계산을 위해 1d로 변경
        trg = trg[1:].view(-1) # loss 계산을 위해 1d로 변경

        loss = criterion(output, trg)
        
        loss.backward()
        # 기울기 clip
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [33]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
      
        for i, batch in enumerate(tqdm(iterator)):
            src = batch.src
            trg = batch.trg

            # output: [trg len, batch size, output dim]
            output = model(src, trg, 0) # teacher forcing off
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim) # [(trg len -1) * batch size, output dim]
            trg = trg[1:].view(-1) # [(trg len -1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [34]:
num_epochs = 10
clip = 1

best_valid_loss = float('inf')

for epoch in range(num_epochs):
    
    train_loss = train(model, train_iterator, optimizer, criterion, clip)
    valid_loss = evaluate(model, valid_iterator, criterion)
      
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

  0%|          | 0/227 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

	Train Loss: 4.421 | Train PPL:  83.175
	 Val. Loss: 3.908 |  Val. PPL:  49.798


  0%|          | 0/227 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

	Train Loss: 1.426 | Train PPL:   4.161
	 Val. Loss: 3.196 |  Val. PPL:  24.428


  0%|          | 0/227 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

	Train Loss: 1.314 | Train PPL:   3.720
	 Val. Loss: 3.240 |  Val. PPL:  25.544


In [35]:
# best val loss일 때의 가중치를 불러옵니다.
model.load_state_dict(torch.load('tut1-model.pt'))

# test loss를 측정합니다.
test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

  0%|          | 0/8 [00:00<?, ?it/s]

| Test Loss: 3.038 | Test PPL:  20.863 |
